In [47]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [92]:
train_datagen=ImageDataGenerator(
                rescale=1.0/255.0,
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                fill_mode="nearest",
                validation_split=0.2
                )
df_train="/kaggle/input/pistachio-image-dataset/Pistachio_Image_Dataset/Pistachio_Image_Dataset"
data_train=train_datagen.flow_from_directory(df_train,
                                      target_size=(256,256),
                                      batch_size=32,
                                      class_mode='binary',
                                      subset='training')
data_val=train_datagen.flow_from_directory(df_train,
                                      target_size=(256,256),
                                      batch_size=32,
                                      class_mode='binary',
                                      subset="validation")


Found 1719 images belonging to 2 classes.
Found 429 images belonging to 2 classes.


In [49]:
data_train.n

1719

In [85]:
img2,lab2=next(data_train)

In [88]:
img2.shape

(32, 256, 256, 3)

In [50]:
def expand_dim(image,labels):
    labels=tf.expand_dims(labels,axis=-1)
    return image,labels


In [51]:
# new_data = map(expand_dim,data)


In [52]:
# new_data

In [53]:
# new_data = [expand_dim(img, lbl) for img, lbl in data]

# for image, labels in new_data:
#     print(image.shape)
#     print(labels.shape)


In [54]:
# for image,labels in new_data:
#     print(image.shape)
#     print(labels.shape)

In [82]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, GlobalAveragePooling2D,Dense,BatchNormalization,Dropout,LeakyReLU,Normalization
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.initializers import HeNormal

In [56]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [57]:
inputs=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation="relu",padding='same')(inputs)
B=BatchNormalization()(C)
D=Dropout(0.25)(B)
C=Conv2D(64,(3,3),activation="relu",padding='same')(D)
B=BatchNormalization()(C)
D=Dropout(0.25)(B)
C=Conv2D(128,(3,3),strides=(2,2),activation="relu",padding='same')(D)
B=BatchNormalization()(C)
D=Dropout(0.25)(B)
C=Conv2D(256,(3,3),strides=(2,2),activation="relu",padding='same')(D)
B=BatchNormalization()(C)
# D=Dropout(0.25)(B)
m=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)
m=GlobalAveragePooling2D()(m)
d=Dense(32,activation='relu')(m)
outputs=Dense(1,activation="sigmoid")(d)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 64, 64, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_8      │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 398,593 (1.52 MB)

 Trainable params: 397,633 (1.52 MB)

 Non-trainable params: 960 (3.75 KB)

In [58]:
model.fit(data_train,epochs=150,validation_data=data_val,callbacks=[early_stopping])

Epoch 1/150


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


54/54 ━━━━━━━━━━━━━━━━━━━━ 89s 1s/step - accuracy: 0.6309 - loss: 0.6336 - val_accuracy: 0.4266 - val_loss: 3.9833
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 44s 737ms/step - accuracy: 0.6987 - loss: 0.5510 - val_accuracy: 0.5012 - val_loss: 0.6962
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 44s 724ms/step - accuracy: 0.7033 - loss: 0.5636 - val_accuracy: 0.6993 - val_loss: 0.5930
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 44s 724ms/step - accuracy: 0.7320 - loss: 0.5089 - val_accuracy: 0.4266 - val_loss: 2.1522
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 44s 730ms/step - accuracy: 0.7675 - loss: 0.4882 - val_accuracy: 0.6387 - val_loss: 0.5644
Epoch 6/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 44s 729ms/step - accuracy: 0.7805 - loss: 0.4541 - val_accuracy: 0.4266 - val_loss: 2.0398
Epoch 7/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 44s 733ms/step - accuracy: 0.7905 - loss: 0.4549 - val_accuracy: 0.7506 - val_loss: 0.4712
Epoch 8/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 45s 742ms/step - accuracy: 0.7732 - loss: 0.4799 - val_accuracy: 0

In [64]:
loss,accuracy=model.evaluate(data_val)


14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 595ms/step - accuracy: 0.5429 - loss: 2.5253


# # m2

In [70]:
inputs2=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation="relu",padding='same')(inputs2)
B=BatchNormalization()(C)
D=Dropout(0.25)(B)
C=Conv2D(64,(3,3),activation="relu",padding='same')(D)
B=BatchNormalization()(C)
D=Dropout(0.25)(B)
C=Conv2D(128,(3,3),strides=(2,2),activation="relu",padding='same')(D)
B=BatchNormalization()(C)
D=Dropout(0.25)(B)
C=Conv2D(256,(3,3),strides=(2,2),activation="relu",padding='same')(D)
B=BatchNormalization()(C)
D=Dropout(0.25)(B)
C=Conv2D(556,(3,3),strides=(2,2),activation="relu",padding='same')(D)
B=BatchNormalization()(C)
# D=Dropout(0.25)(B)
m=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)
m=GlobalAveragePooling2D()(m)
d=Dense(32,activation='relu')(m)
outputs2=Dense(1,activation="sigmoid")(d)
model2=Model(inputs2,outputs2)
model2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model2.summary()

Model: "functional_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_41 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_33          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_42 (Conv2D)              │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_34          │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_43 (Conv2D)              │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_35          │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_44 (Conv2D)              │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_36          │ (None, 64, 64, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_30 (Dropout)            │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_45 (Conv2D)              │ (None, 32, 32, 556)    │     1,281,580 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_37          │ (None, 32, 32, 556)    │         2,224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 16, 16, 556)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_13     │ (None, 556)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 32)             │        17,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,691,997 (6.45 MB)

 Trainable params: 1,689,925 (6.45 MB)

 Non-trainable params: 2,072 (8.09 KB)

In [71]:
model2.fit(data_train,epochs=150,validation_data=data_val,callbacks=[early_stopping])

Epoch 1/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.6426 - loss: 0.6377 - val_accuracy: 0.4266 - val_loss: 23.9853
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 45s 742ms/step - accuracy: 0.7107 - loss: 0.5561 - val_accuracy: 0.4266 - val_loss: 9.2143
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 45s 760ms/step - accuracy: 0.7418 - loss: 0.5123 - val_accuracy: 0.4312 - val_loss: 1.0173
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 44s 741ms/step - accuracy: 0.7402 - loss: 0.5022 - val_accuracy: 0.6177 - val_loss: 0.8165
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 45s 744ms/step - accuracy: 0.7724 - loss: 0.4897 - val_accuracy: 0.5734 - val_loss: 9.0109
Epoch 5: early stopping


In [72]:
loss,accuracy=model2.evaluate(data_val)


14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 566ms/step - accuracy: 0.5825 - loss: 8.7100


# # m3

In [75]:
inputs3=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation="relu",padding='same')(inputs3)
B=BatchNormalization()(C)
D=Dropout(0.25)(B)
C=Conv2D(64,(3,3),activation="relu",padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
B=BatchNormalization()(C)
D=Dropout(0.25)(B)
C=Conv2D(128,(3,3),strides=(2,2),activation="relu",padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
B=BatchNormalization()(C)
D=Dropout(0.25)(B)
C=Conv2D(256,(3,3),strides=(2,2),activation="relu",padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
B=BatchNormalization()(C)
D=Dropout(0.25)(B)
C=Conv2D(556,(3,3),strides=(2,2),activation="relu",padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
B=BatchNormalization()(C)
# D=Dropout(0.25)(B)
m=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)
m=GlobalAveragePooling2D()(m)
d=Dense(32,activation='relu')(m)
outputs3=Dense(1,activation="sigmoid")(d)
model3=Model(inputs3,outputs3)
model3.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model3.summary()

Model: "functional_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_16 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_47 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_39          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_48 (Conv2D)              │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_40          │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_33 (Dropout)            │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_49 (Conv2D)              │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_41          │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_34 (Dropout)            │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_50 (Conv2D)              │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_42          │ (None, 64, 64, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_35 (Dropout)            │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_51 (Conv2D)              │ (None, 32, 32, 556)    │     1,281,580 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_43          │ (None, 32, 32, 556)    │         2,224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 16, 16, 556)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_14     │ (None, 556)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 32)             │        17,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,691,997 (6.45 MB)

 Trainable params: 1,689,925 (6.45 MB)

 Non-trainable params: 2,072 (8.09 KB)

In [76]:
model3.fit(data_train,epochs=150,validation_data=data_val,callbacks=[early_stopping])

Epoch 1/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 71s 921ms/step - accuracy: 0.6093 - loss: 14.3046 - val_accuracy: 0.4266 - val_loss: 5.5333
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 45s 741ms/step - accuracy: 0.6970 - loss: 2.1441 - val_accuracy: 0.5734 - val_loss: 1.1496
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 48s 801ms/step - accuracy: 0.7061 - loss: 0.9108 - val_accuracy: 0.5082 - val_loss: 0.9159
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 50s 831ms/step - accuracy: 0.7089 - loss: 0.7156 - val_accuracy: 0.5734 - val_loss: 1.0846
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 48s 796ms/step - accuracy: 0.7099 - loss: 0.6660 - val_accuracy: 0.6573 - val_loss: 0.7196
Epoch 5: early stopping


In [77]:
loss,accuracy=model3.evaluate(data_val)


14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 661ms/step - accuracy: 0.6813 - loss: 0.7003


# # m4

In [80]:
inputs4=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation="relu",padding='same')(inputs4)
B=BatchNormalization()(C)
D=Dropout(0.25)(B)
C=Conv2D(64,(3,3),activation=None,padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
R=LeakyReLU(0.08)(C)
B=BatchNormalization()(R)
D=Dropout(0.25)(B)
C=Conv2D(128,(5,5),strides=(2,2),activation=None,padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
R=LeakyReLU(0.08)(C)

B=BatchNormalization()(R)
D=Dropout(0.25)(B)
C=Conv2D(256,(5,5),strides=(2,2),activation=None,padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
R=LeakyReLU(0.08)(C)

B=BatchNormalization()(R)
D=Dropout(0.25)(B)
C=Conv2D(556,(5,5),strides=(2,2),activation=None,padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
R=LeakyReLU(0.08)(C)

B=BatchNormalization()(R)
# D=Dropout(0.25)(B)
m=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)
m=GlobalAveragePooling2D()(m)
d=Dense(32,activation='relu')(m)
outputs4=Dense(1,activation="sigmoid")(d)
model4=Model(inputs4,outputs4)
model4.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model4.summary()

Model: "functional_31"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_18 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_54 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_45          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_37 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_55 (Conv2D)              │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_46          │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_38 (Dropout)            │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_56 (Conv2D)              │ (None, 128, 128, 128)  │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_47          │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_39 (Dropout)            │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_57 (Conv2D)              │ (None, 64, 64, 256)    │       819,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_48          │ (None, 64, 64, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_40 (Dropout)            │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_58 (Conv2D)              │ (None, 32, 32, 556)    │     3,558,956 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 32, 32, 556)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_49          │ (None, 32, 32, 556)    │         2,224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 16, 16, 556)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_15     │ (None, 556)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 32)             │        17,82

 Total params: 4,624,733 (17.64 MB)

 Trainable params: 4,622,661 (17.63 MB)

 Non-trainable params: 2,072 (8.09 KB)

In [83]:
model4.fit(data_train,epochs=150,validation_data=data_val,callbacks=[early_stopping])

Epoch 1/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 130s 2s/step - accuracy: 0.6476 - loss: 14.4886 - val_accuracy: 0.4266 - val_loss: 3.3222
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 53s 893ms/step - accuracy: 0.6804 - loss: 1.9059 - val_accuracy: 0.4266 - val_loss: 1.7313
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 53s 891ms/step - accuracy: 0.6997 - loss: 0.9523 - val_accuracy: 0.5734 - val_loss: 0.9815
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 53s 892ms/step - accuracy: 0.6909 - loss: 0.8187 - val_accuracy: 0.5734 - val_loss: 1.9718
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 53s 892ms/step - accuracy: 0.7081 - loss: 0.7578 - val_accuracy: 0.6200 - val_loss: 0.8983
Epoch 5: early stopping


In [84]:
loss,accuracy=model4.evaluate(data_val)


14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 586ms/step - accuracy: 0.6345 - loss: 0.8727


# # m5

In [89]:
inputs5=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation="relu",padding='same')(inputs5)
B=BatchNormalization()(C)
D=Dropout(0.25)(B)
C=Conv2D(64,(3,3),activation=None,padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
R=LeakyReLU(0.08)(C)
B=BatchNormalization()(R)
D=Dropout(0.25)(B)
C=Conv2D(128,(5,5),activation=None,padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
R=LeakyReLU(0.08)(C)
M=MaxPooling2D(2,2)(R)
B=BatchNormalization()(M)
D=Dropout(0.25)(B)
C=Conv2D(256,(5,5),activation=None,padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
R=LeakyReLU(0.08)(C)
M=MaxPooling2D(2,2)(R)

B=BatchNormalization()(M)
D=Dropout(0.25)(B)
C=Conv2D(556,(5,5),activation=None,padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
R=LeakyReLU(0.08)(C)
M=MaxPooling2D(2,2)(R)

B=BatchNormalization()(M)
# D=Dropout(0.25)(B)
# m=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)
m=GlobalAveragePooling2D()(B)
d=Dense(32,activation='relu')(m)
outputs5=Dense(1,activation="sigmoid")(d)
model5=Model(inputs5,outputs5)
model5.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model5.summary()

Model: "functional_33"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_19 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_59 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_50          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_41 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_60 (Conv2D)              │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_51          │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_42 (Dropout)            │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_61 (Conv2D)              │ (None, 256, 256, 128)  │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 256, 256, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_52          │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_43 (Dropout)            │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_62 (Conv2D)              │ (None, 128, 128, 256)  │       819,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_6 (LeakyReLU)       │ (None, 128, 128, 256)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_53          │ (None, 64, 64, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_44 (Dropout)            │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_63 (Conv2D)              │ (None, 64, 64, 556)    │     3,558,956 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_7 (LeakyReLU)       │ (None, 64, 64, 556)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 32, 32, 556)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_54          │ (None, 32, 32, 556)    │         2,224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 4,624,733 (17.64 MB)

 Trainable params: 4,622,661 (17.63 MB)

 Non-trainable params: 2,072 (8.09 KB)

In [90]:
model5.fit(data_train,epochs=150,validation_data=data_val,callbacks=[early_stopping])

Epoch 1/150


2024-09-27 13:47:47.438428: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng11{k2=3,k3=0} for conv (f32[32,128,256,256]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,64,256,256]{3,2,1,0}, f32[128,64,5,5]{3,2,1,0}, f32[128]{0}), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-09-27 13:47:47.706262: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.267970351s
Trying algorithm eng11{k2=3,k3=0} for conv (f32[32,128,256,256]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,64,256,256]{3,2,1,0}, f32[128,64,5,5]{3,2,1,0}, f32[128]{0}), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"

34/54 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.6602 - loss: 13.3995

2024-09-27 13:50:04.133055: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng36{k2=3,k3=0} for conv (f32[23,128,256,256]{3,2,1,0}, u8[0]{0}) custom-call(f32[23,64,256,256]{3,2,1,0}, f32[128,64,5,5]{3,2,1,0}, f32[128]{0}), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-09-27 13:50:04.526054: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.393096527s
Trying algorithm eng36{k2=3,k3=0} for conv (f32[23,128,256,256]{3,2,1,0}, u8[0]{0}) custom-call(f32[23,64,256,256]{3,2,1,0}, f32[128,64,5,5]{3,2,1,0}, f32[128]{0}), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"

54/54 ━━━━━━━━━━━━━━━━━━━━ 265s 3s/step - accuracy: 0.6684 - loss: 10.9713 - val_accuracy: 0.4266 - val_loss: 6.6381
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step - accuracy: 0.7153 - loss: 1.2110 - val_accuracy: 0.4289 - val_loss: 2.1378
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step - accuracy: 0.6904 - loss: 0.8681 - val_accuracy: 0.5734 - val_loss: 1.0468
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step - accuracy: 0.6997 - loss: 0.8419 - val_accuracy: 0.5944 - val_loss: 1.5818
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step - accuracy: 0.7286 - loss: 0.7927 - val_accuracy: 0.7855 - val_loss: 0.7472
Epoch 5: early stopping


In [91]:
loss,accuracy=model5.evaluate(data_val)


14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 588ms/step - accuracy: 0.7264 - loss: 0.7730


In [68]:
# for layer in model.layers:
#     layer.trainable=False
# model_output=model.layer[-4].output

# # m6

In [95]:
inputs6=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation="relu",padding='same')(inputs6)
B=BatchNormalization()(C)
D=Dropout(0.25)(B)
C=Conv2D(64,(3,3),activation=None,padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
R=LeakyReLU(0.08)(C)
B=BatchNormalization()(R)
D=Dropout(0.25)(B)
C=Conv2D(128,(5,5),activation=None,padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
R=LeakyReLU(0.08)(C)
M=MaxPooling2D(2,2)(R)
B=BatchNormalization()(M)
D=Dropout(0.25)(B)
C=Conv2D(256,(7,7),activation=None,padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
R=LeakyReLU(0.08)(C)
M=MaxPooling2D(2,2)(R)

B=BatchNormalization()(M)
D=Dropout(0.25)(B)
C=Conv2D(556,(11,11),activation=None,padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
R=LeakyReLU(0.08)(C)
M=MaxPooling2D(2,2)(R)

B=BatchNormalization()(M)
# D=Dropout(0.25)(B)
# m=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)
m=GlobalAveragePooling2D()(B)
d=Dense(32,activation='relu')(m)
outputs6=Dense(1,activation="sigmoid")(d)
model6=Model(inputs6,outputs6)
model6.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model6.summary()

Model: "functional_37"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_21 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_69 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_60          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_49 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_70 (Conv2D)              │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_12 (LeakyReLU)      │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_61          │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_50 (Dropout)            │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_71 (Conv2D)              │ (None, 256, 256, 128)  │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_13 (LeakyReLU)      │ (None, 256, 256, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_62          │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_51 (Dropout)            │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_72 (Conv2D)              │ (None, 128, 128, 256)  │     1,605,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_14 (LeakyReLU)      │ (None, 128, 128, 256)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_63          │ (None, 64, 64, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_52 (Dropout)            │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_73 (Conv2D)              │ (None, 64, 64, 556)    │    17,223,212 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_15 (LeakyReLU)      │ (None, 64, 64, 556)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 32, 32, 556)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_64          │ (None, 32, 32, 556)    │         2,224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 19,075,421 (72.77 MB)

 Trainable params: 19,073,349 (72.76 MB)

 Non-trainable params: 2,072 (8.09 KB)

In [96]:
model6.fit(data_train,epochs=150,validation_data=data_val,callbacks=[early_stopping])

Epoch 1/150


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-09-27 14:29:14.372579: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng20{k2=1,k4=3,k5=1,k6=0,k7=0,k19=0} for conv (f32[32,256,128,128]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,128,128,128]{3,2,1,0}, f32[256,128,7,7]{3,2,1,0}, f32[256]{0}), window={size=7x7 pad=3_3x3_3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-09-27 14:29:14.4574

13/54 ━━━━━━━━━━━━━━━━━━━━ 2:28 4s/step - accuracy: 0.5717 - loss: 16.3955

2024-09-27 14:32:20.433394: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng11{k2=2,k3=0} for conv (f32[23,256,128,128]{3,2,1,0}, u8[0]{0}) custom-call(f32[23,128,128,128]{3,2,1,0}, f32[256,128,7,7]{3,2,1,0}, f32[256]{0}), window={size=7x7 pad=3_3x3_3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-09-27 14:32:20.674996: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.241723264s
Trying algorithm eng11{k2=2,k3=0} for conv (f32[23,256,128,128]{3,2,1,0}, u8[0]{0}) custom-call(f32[23,128,128,128]{3,2,1,0}, f32[256,128,7,7]{3,2,1,0}, f32[256]{0}), window={size=7x7 pad=3_3x3_3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_confi

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.6341 - loss: 9.7740

2024-09-27 14:36:46.184147: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng20{k2=1,k4=3,k5=1,k6=0,k7=0,k19=0} for conv (f32[13,556,64,64]{3,2,1,0}, u8[0]{0}) custom-call(f32[13,256,64,64]{3,2,1,0}, f32[556,256,11,11]{3,2,1,0}, f32[556]{0}), window={size=11x11 pad=5_5x5_5}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-09-27 14:36:46.412783: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.228741556s
Trying algorithm eng20{k2=1,k4=3,k5=1,k6=0,k7=0,k19=0} for conv (f32[13,556,64,64]{3,2,1,0}, u8[0]{0}) custom-call(f32[13,256,64,64]{3,2,1,0}, f32[556,256,11,11]{3,2,1,0}, f32[556]{0}), window={size=11x11 pad=5_5x5_5}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn

54/54 ━━━━━━━━━━━━━━━━━━━━ 469s 6s/step - accuracy: 0.6349 - loss: 9.6910 - val_accuracy: 0.5734 - val_loss: 2.7037
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 221s 4s/step - accuracy: 0.6990 - loss: 1.3167 - val_accuracy: 0.5734 - val_loss: 4.0189
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 221s 4s/step - accuracy: 0.7090 - loss: 1.0644 - val_accuracy: 0.5734 - val_loss: 2.0658
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 221s 4s/step - accuracy: 0.6804 - loss: 0.9900 - val_accuracy: 0.5734 - val_loss: 2.4134
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 221s 4s/step - accuracy: 0.7130 - loss: 0.9346 - val_accuracy: 0.5734 - val_loss: 2.6168
Epoch 5: early stopping


In [98]:
loss,accuracy=model6.evaluate(data_val)


14/14 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.5840 - loss: 2.5614


# # m7

In [101]:
inputs7=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation="relu",padding='same')(inputs7)
B=BatchNormalization()(C)
D=Dropout(0.25)(B)
C=Conv2D(64,(3,3),activation=None,padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
R=LeakyReLU(0.08)(C)
B=BatchNormalization()(R)
D=Dropout(0.25)(B)
C=Conv2D(128,(5,5),activation=None,padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
R=LeakyReLU(0.08)(C)
M=MaxPooling2D(2,2)(R)
B=BatchNormalization()(M)
D=Dropout(0.25)(B)
C=Conv2D(256,(5,5),activation=None,padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
R=LeakyReLU(0.08)(C)
M=MaxPooling2D(2,2)(R)

B=BatchNormalization()(M)
D=Dropout(0.25)(B)
C=Conv2D(556,(5,5),activation=None,padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
R=LeakyReLU(0.08)(C)
M=MaxPooling2D(2,2)(R)

B=BatchNormalization()(M)
D=Dropout(0.25)(B)

C=Conv2D(1024,(5,5),activation=None,padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
R=LeakyReLU(0.08)(C)
M=MaxPooling2D(2,2)(R)

B=BatchNormalization()(M)
# D=Dropout(0.25)(B)
# m=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)
m=GlobalAveragePooling2D()(B)
d=Dense(32,activation='relu')(m)
outputs7=Dense(1,activation="sigmoid")(d)
model7=Model(inputs7,outputs7)
model7.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model7.summary()

Model: "functional_41"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_23 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_80 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_71          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_58 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_81 (Conv2D)              │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_21 (LeakyReLU)      │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_72          │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_59 (Dropout)            │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_82 (Conv2D)              │ (None, 256, 256, 128)  │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_22 (LeakyReLU)      │ (None, 256, 256, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_73          │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_60 (Dropout)            │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_83 (Conv2D)              │ (None, 128, 128, 256)  │       819,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_23 (LeakyReLU)      │ (None, 128, 128, 256)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_27 (MaxPooling2D) │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_74          │ (None, 64, 64, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_61 (Dropout)            │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_84 (Conv2D)              │ (None, 64, 64, 556)    │     3,558,956 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_24 (LeakyReLU)      │ (None, 64, 64, 556)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 32, 32, 556)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_75          │ (None, 32, 32, 556)    │         2,224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 18,878,429 (72.02 MB)

 Trainable params: 18,874,309 (72.00 MB)

 Non-trainable params: 4,120 (16.09 KB)

In [102]:
model7.fit(data_train,epochs=150,validation_data=data_val,callbacks=[early_stopping])

Epoch 1/150


2024-09-27 14:54:19.552026: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng25{k2=2,k3=0} for conv (f32[32,556,32,32]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,1024,32,32]{3,2,1,0}, f32[1024,556,5,5]{3,2,1,0}), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-09-27 14:54:20.195269: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.643369016s
Trying algorithm eng25{k2=2,k3=0} for conv (f32[32,556,32,32]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,1024,32,32]{3,2,1,0}, f32[1024,556,5,5]{3,2,1,0}), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"operation_queue_id":"0","wait_on_operation

34/54 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.6620 - loss: 24.9461

2024-09-27 14:55:59.972327: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng25{k2=2,k3=0} for conv (f32[23,556,32,32]{3,2,1,0}, u8[0]{0}) custom-call(f32[23,1024,32,32]{3,2,1,0}, f32[1024,556,5,5]{3,2,1,0}), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-09-27 14:56:00.146237: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.17402028s
Trying algorithm eng25{k2=2,k3=0} for conv (f32[23,556,32,32]{3,2,1,0}, u8[0]{0}) custom-call(f32[23,1024,32,32]{3,2,1,0}, f32[1024,556,5,5]{3,2,1,0}), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"operation_queue_id":"0","wait_on_operation_

54/54 ━━━━━━━━━━━━━━━━━━━━ 207s 3s/step - accuracy: 0.6668 - loss: 20.0185 - val_accuracy: 0.5734 - val_loss: 2.6464
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 121s 2s/step - accuracy: 0.7067 - loss: 1.5594 - val_accuracy: 0.5734 - val_loss: 2.0310
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 121s 2s/step - accuracy: 0.6847 - loss: 1.1552 - val_accuracy: 0.5734 - val_loss: 3.0149
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 121s 2s/step - accuracy: 0.6871 - loss: 1.0642 - val_accuracy: 0.5734 - val_loss: 2.5033
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 123s 2s/step - accuracy: 0.6951 - loss: 1.2013 - val_accuracy: 0.5734 - val_loss: 3.8729
Epoch 5: early stopping


In [103]:
loss,accuracy=model7.evaluate(data_val)

14/14 ━━━━━━━━━━━━━━━━━━━━ 11s 751ms/step - accuracy: 0.5623 - loss: 3.9623
